In [1]:
# Import libraries
import ee
ee.Initialize()
import geemap

In [2]:

# Define the study area
study_area = ee.FeatureCollection('users/raulperezastorga/Limites_SPPP')

#######Temporada incendios inicia en agosto y termina en Julio
t1='2023'
t2='2024'
t1i=t1 + '-07-01'
t1f=t1 + '-11-30'
t2i=t1 +'-12-01'
t2f=t2 + '-06-30'

# Mask active vegetation
def calc_nbr(image):
    ndvi = image.normalizedDifference(['B4', 'B7'])
    return image.addBands(ndvi.rename('nbr'))

def mask_active_veg(image):
    ndvi = image.select('NDVI')
    veg_mask = ndvi.lt(0.1)
    return image.updateMask(veg_mask)

# Filter fires by FIRMS dataset
collection_firms = ee.ImageCollection("MODIS/061/MCD64A1") \
    .filterBounds(study_area) \
    .filterDate(t1f, t2f) \
    .select('BurnDate')
    
fire_mask = collection_firms.max().gte(0).rename('fires')

def mask_fires(image):
    return image.updateMask(fire_mask)

elevation_data = ee.Image('USGS/SRTMGL1_003').select('elevation').clip(study_area)
elevation_mask = elevation_data.lt(3000)

# define a function that masks clouds in Landsat 5 images
def maskL5sr(image):
      # create a cloud mask for Landsat 5 SR data
      cloudShadowBitMask = (1 << 3)
      cloudsBitMask = (1 << 5)
      qa = image.select('pixel_qa')
      mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
            qa.bitwiseAnd(cloudsBitMask).eq(0))
      return image.updateMask(mask)


# Load Landsat 5 collection
landsat_t1 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
    .filterBounds(study_area) \
    .filterDate(t1i, t1f)\
    .map(mask_fires)\
    .map(calc_nbr)\
    .map(lambda img: img.updateMask(elevation_mask))\
    .map(maskL5sr)\
    .filterMetadata('CLOUD_COVER','less_than',10)
    



# Load Landsat 5 collection
landsat_t2 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
    .filterBounds(study_area) \
    .filterDate(t2i, t2f)\
    .map(mask_fires)\
    .map(calc_nbr)\
    .map(lambda img: img.updateMask(elevation_mask))\
    .map(maskL5sr)\
    .filterMetadata('CLOUD_COVER','less_than',10)

preNBR = landsat_t1.select('nbr').max()
postNBR = landsat_t2.select('nbr').min()

dNBR_unscaled = preNBR.subtract(postNBR)
#dNBR_unscaled=dNBR_unscaled.clip(r20)
dNBR_reclass = dNBR_unscaled.gte(-0.5).add(dNBR_unscaled.gte(-0.25)).add(dNBR_unscaled.gte(-0.1)).add(dNBR_unscaled.gte(0.1)).add(dNBR_unscaled.gte(0.27)).add(dNBR_unscaled.gte(0.44)).add(dNBR_unscaled.gte(0.66))
#dNBR_Seguro = dNBR_unscaled.gte(0.27)

viz_dNBR={
  'min':1,
  'max':7,#"#7a8737","#acbe4d","#0ae042",
  'palette':["#7a8737","#acbe4d","#0ae042","#fff70b","#ffaf38","#ff641b","#a41fd6"]
}


# Define visualization parameters for false color composite
viz_params = {
  'bands': ['B3', 'B2', 'B1'],
  'min': 0,
  'max': 3000,
  'gamma':1.3
}


Map = geemap.Map()
Map


# Print the resulting collection



Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [3]:
Map.addLayer(dNBR_reclass, viz_dNBR, 'dNBR Clase')
Map.addLayer(landsat_t2, viz_params, 'NBR T2')
Map.addLayer(landsat_t1, viz_params, 'NBR T1')

EEException: Image.gte: If one image has no bands, the other must also have no bands. Got 0 and 1.